# RSA (Rivest-Shamir-Adleman) Asymmetric Encryption

## Overview

This notebook demonstrates RSA asymmetric encryption with educational examples.

### Asymmetric Encryption

The keys are defined from two primes p and q:
- n = p × q
- Typical modulus sizes include 1024, 2048 and 4096 bits (implementation-dependent)

### RSA Flow

- A generates a key pair: public (Ka+) and private (Ka-)
- B generates a key pair: public (Kb+) and private (Kb-)
- A and B may publish or send their public keys to a trusted third party (TTP)
- The TTP can distribute certificates or key data (Ca, Cb) as needed
- To send M to B, A computes the ciphertext C = E(Kb+, M)
- B recovers M by computing M = D(Kb-, C)

### RSA Key Setup

1. Choose two distinct large primes p and q (keep them secret)
2. Compute n = p × q. The modulus n is used in both public and private keys
3. Compute Euler's totient: φ(n) = (p - 1) × (q - 1)
4. Select an encryption exponent e with 1 < e < φ(n) and gcd(e, φ(n)) = 1
5. Compute d as the modular inverse of e modulo φ(n): e × d ≡ 1 (mod φ(n))
6. Public key: K+ = {e, n}. Private key: K- = {d, n}

### Example Key Setup (Small Numbers)

For p = 17 and q = 11:
- n = p × q = 17 × 11 = 187
- φ(n) = (p - 1) × (q - 1) = 16 × 10 = 160
- Select e = 7 (valid since gcd(7, 160) = 1)
- Compute d = 23 (since 7 × 23 = 161 ≡ 1 (mod 160))

Therefore:
- Public key K+ = {e, n} = {7, 187}
- Private key K- = {d, n} = {23, 187}

### Encryption/Decryption

If M = 88, then (textbook RSA):
- **Encryption:** C = M^e mod n
- **Decryption:** M = C^d mod n

---

**⚠️ Important:** Real RSA uses large primes (2048+ bits) and secure padding (e.g., OAEP). Do not use textbook RSA in production!

## 1. Helper Functions

In [20]:
from typing import Tuple
from math import gcd


def egcd(a: int, b: int, depth: int = 0) -> Tuple[int, int, int]:
    """Extended Euclidean Algorithm with detailed logging.
    
    Returns a tuple (g, x, y) such that a*x + b*y = g = gcd(a, b).
    """
    indent = "  " * depth
    
    print(f"{indent}🔍 egcd({a}, {b})")
    
    if b == 0:
        print(f"{indent}✓ Base case: b = 0")
        print(f"{indent}  Returning: gcd = {a}, x = 1, y = 0")
        print(f"{indent}  Verification: {a}×1 + 0×0 = {a}")
        return (a, 1, 0)
    
    # Show the division
    quotient = a // b
    remainder = a % b
    print(f"{indent}  Division: {a} = {b} × {quotient} + {remainder}")
    print(f"{indent}  Recursing with ({b}, {remainder})...")
    
    # Recursive call
    g, x1, y1 = egcd(b, remainder, depth + 1)
    
    # Backtracking computation
    x = y1
    y = x1 - quotient * y1
    
    print(f"{indent}⬅️  Backtracking from egcd({b}, {remainder})")
    print(f"{indent}  Got: g = {g}, x1 = {x1}, y1 = {y1}")
    print(f"{indent}  Computing: x = y1 = {y1}")
    print(f"{indent}  Computing: y = x1 - ({a}//{b}) × y1 = {x1} - {quotient} × {y1} = {y}")
    print(f"{indent}  Verification: {a}×{x} + {b}×{y} = {a*x + b*y} = {g} ✓")
    
    return (g, x, y)


def modinv(a: int, m: int, verbose: bool = True) -> int:
    """Modular inverse with detailed logging: find x such that (a * x) % m == 1.
    
    Raises ValueError if the inverse does not exist.
    """
    if verbose:
        print(f"\n{'='*60}")
        print(f"COMPUTING MODULAR INVERSE")
        print(f"{'='*60}")
        print(f"Goal: Find x such that ({a} × x) ≡ 1 (mod {m})")
        print(f"      Or equivalently: ({a} × x) mod {m} = 1")
        print(f"\nStep 1: Use Extended Euclidean Algorithm")
        print(f"        We need {a} × x + {m} × y = gcd({a}, {m})")
        print()
    
    g, x, y = egcd(a, m)
    
    if verbose:
        print(f"\n{'─'*60}")
        print(f"Step 2: Check if modular inverse exists")
        print(f"        gcd({a}, {m}) = {g}")
    
    if g != 1:
        if verbose:
            print(f"        ✗ Inverse does NOT exist (gcd ≠ 1)")
            print(f"{'='*60}\n")
        raise ValueError(f"modular inverse does not exist for {a} modulo {m}")
    
    if verbose:
        print(f"        ✓ Inverse EXISTS (gcd = 1)")
        print(f"\nStep 3: Extract the modular inverse")
        print(f"        From egcd: {a} × {x} + {m} × {y} = 1")
        print(f"        Taking mod {m}: {a} × {x} ≡ 1 (mod {m})")
        print(f"        Therefore: x = {x}")
        
        # Normalize to positive
        result = x % m
        if x != result:
            print(f"\nStep 4: Normalize to positive value")
            print(f"        x mod {m} = {x} mod {m} = {result}")
        else:
            print(f"\nStep 4: Already positive, x = {result}")
        
        # Verification
        print(f"\n✅ VERIFICATION:")
        print(f"   ({a} × {result}) mod {m} = {(a * result) % m}")
        print(f"   Expected: 1")
        print(f"   {'✓ CORRECT' if (a * result) % m == 1 else '✗ ERROR'}")
        print(f"{'='*60}\n")
    
    return x % m


def modinv_simple(a: int, m: int) -> int:
    """Modular inverse without logging (for production use)."""
    g, x, _ = egcd(a, m, depth=0)
    if g != 1:
        raise ValueError(f"modular inverse does not exist for {a} modulo {m}")
    return x % m


# Test the functions
if __name__ == "__main__":
    print("\n" + "="*60)
    print("EXAMPLE 1: Computing modular inverse of 7 modulo 160")
    print("="*60)
    result = modinv(7, 160)
    print(f"Result: 7^(-1) ≡ {result} (mod 160)")
    
    print("\n" + "="*60)
    print("EXAMPLE 2: Computing modular inverse of 3 modulo 11")
    print("="*60)
    result = modinv(3, 11)
    print(f"Result: 3^(-1) ≡ {result} (mod 11)")
    
    print("\n" + "="*60)
    print("EXAMPLE 3: Attempting inverse that doesn't exist")
    print("="*60)
    try:
        result = modinv(6, 9)
    except ValueError as e:
        print(f"Caught expected error: {e}")


EXAMPLE 1: Computing modular inverse of 7 modulo 160

COMPUTING MODULAR INVERSE
Goal: Find x such that (7 × x) ≡ 1 (mod 160)
      Or equivalently: (7 × x) mod 160 = 1

Step 1: Use Extended Euclidean Algorithm
        We need 7 × x + 160 × y = gcd(7, 160)

🔍 egcd(7, 160)
  Division: 7 = 160 × 0 + 7
  Recursing with (160, 7)...
  🔍 egcd(160, 7)
    Division: 160 = 7 × 22 + 6
    Recursing with (7, 6)...
    🔍 egcd(7, 6)
      Division: 7 = 6 × 1 + 1
      Recursing with (6, 1)...
      🔍 egcd(6, 1)
        Division: 6 = 1 × 6 + 0
        Recursing with (1, 0)...
        🔍 egcd(1, 0)
        ✓ Base case: b = 0
          Returning: gcd = 1, x = 1, y = 0
          Verification: 1×1 + 0×0 = 1
      ⬅️  Backtracking from egcd(1, 0)
        Got: g = 1, x1 = 1, y1 = 0
        Computing: x = y1 = 0
        Computing: y = x1 - (6//1) × y1 = 1 - 6 × 0 = 1
        Verification: 6×0 + 1×1 = 1 = 1 ✓
    ⬅️  Backtracking from egcd(6, 1)
      Got: g = 1, x1 = 0, y1 = 1
      Computing: x = y1 = 1
  

## 2. RSA Key Setup

In [19]:
def Key_Setup(p, q):
    """Key setup for RSA"""
    print(f"\n{'='*60}")
    print(f"RSA KEY SETUP")
    print(f"{'='*60}")
    print(f"Primes: p = {p}, q = {q}")
    
    n = p * q
    totient = (p-1) * (q-1)
    print(f"Modulus: n = p × q = {n}")
    print(f"Totient: φ(n) = (p-1)(q-1) = {totient}")
    
    # Find valid e values
    e_values = [e for e in range(2, totient) if gcd(e, totient) == 1]
    e = e_values[0]
    d = modinv(e, totient)
    
    print(f"\nPublic exponent:  e = {e}")
    print(f"Private exponent: d = {d}")
    print(f"{'='*60}\n")
    
    return (e, n), (d, n)


# Test the key setup
Key_Setup(107, 113)


RSA KEY SETUP
Primes: p = 107, q = 113
Modulus: n = p × q = 12091
Totient: φ(n) = (p-1)(q-1) = 11872


NameError: name 'gcd' is not defined

In [17]:
def mod_inverse(a, m):
    """Find modular multiplicative inverse of a mod m using Extended Euclidean Algorithm"""
    def extended_gcd(a, b):
        if a == 0:
            return b, 0, 1
        gcd, x1, y1 = extended_gcd(b % a, a)
        x = y1 - (b // a) * x1
        y = x1
        return gcd, x, y
    
    gcd, x, _ = extended_gcd(a % m, m)
    if gcd != 1:
        raise ValueError(f"Modular inverse does not exist for {a} mod {m}")
    return (x % m + m) % m  # Ensure positive result

In [18]:
def factor_n(n):
    """Factor n into two primes (only works for small n)"""
    valid_tuples = []
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            print(f"Found factors: {i} and {n // i}")
            valid_tuples.append((i, n // i))
    if valid_tuples:
        return valid_tuples[0]
    return None, None

def get_public_key(private_key, n):
    """Derive public key from private key and modulus n"""
    d = private_key
    # Find e only with d and n
    p, q = factor_n(n)
    print(f"Factored n={n} into p={p}, q={q}")
    totient = (p-1)*(q-1)
    print(f"Calculated totient: {totient}")
    e = mod_inverse(d, totient)
    return (e, n)

def get_private_key(public_key, n):
    """Derive private key from public key and modulus n"""
    e = public_key
    p, q = factor_n(n)
    print(f"Factored n={n} into p={p}, q={q}")
    totient = (p-1)*(q-1)
    print(f"Calculated totient: {totient}")
    d = mod_inverse(e, totient)
    return (d, n)

def verify_keys(public_key, private_key, n):
    """Verify that the public and private keys are correct inverses"""
    e = public_key
    d = private_key
    p, q = factor_n(n)
    print(f"Factored n={n} into p={p}, q={q}")
    totient = (p-1)*(q-1)
    print(f"Calculated totient: {totient}")
    result = (e * d) % totient
    print(f"Verifying keys: (e * d) mod φ(n) = ({e} * {d}) mod {totient} = {result}")
    return result == 1

private_key = get_private_key(23, 143)
print(f"Derived private key: {private_key}")
public_key = get_public_key(private_key[0], 143)
print(f"Derived public key: {public_key}")
verify_keys(public_key[0], private_key[0], 143)




Found factors: 11 and 13
Factored n=143 into p=11, q=13
Calculated totient: 120
Derived private key: (47, 143)
Found factors: 11 and 13
Factored n=143 into p=11, q=13
Calculated totient: 120
Derived public key: (23, 143)
Found factors: 11 and 13
Factored n=143 into p=11, q=13
Calculated totient: 120
Verifying keys: (e * d) mod φ(n) = (23 * 47) mod 120 = 1


True

## 3. RSA Encryption and Decryption Functions

In [21]:
# Global variables to store keys
PUBLIC_KEY = None
PRIVATE_KEY = None


def initialize_keys(p=61, q=53):
    """Initialize RSA keys once"""
    global PUBLIC_KEY, PRIVATE_KEY
    PUBLIC_KEY, PRIVATE_KEY = Key_Setup(p, q)

def set_keys(private_key, public_key):
    """Set RSA keys manually"""
    global PUBLIC_KEY, PRIVATE_KEY
    PRIVATE_KEY = private_key
    PUBLIC_KEY = public_key


def RSA_encrypt(M):
    """Encrypt a single block"""
    e, n = PUBLIC_KEY
    C = pow(M, e, n)
    print(f"C = M^e mod n = {M}^{e} mod {n} = {C}")
    return C


def RSA_decrypt(C):
    """Decrypt a single block"""
    d, n = PRIVATE_KEY
    M = pow(C, d, n)
    print(f"M = C^d mod n = {C}^{d} mod {n} = {M}")
    return M


def RSA_encrypt_plaintext(M, A, Space):
    """Encrypt plaintext message"""
    print(f"\n{'='*60}")
    print(f"ENCRYPTING: '{M}'")
    print(f"{'='*60}")
    
    # Encode characters to numbers
    M_number = []
    print("\n📝 Encoding characters:")
    for char in M:
        if char == ' ':
            char_value = Space
            print(f"  ' ' → {char_value:02d}")
        else:
            char_value = ord(char.upper()) - A
            print(f"  '{char}' → {char_value:02d}")
        M_number.append(char_value)
    
    # Create blocks
    block_size = 4
    M_string = ''.join(f"{m:02d}" for m in M_number)
    M_final_blocks = [int(M_string[i:i+block_size]) for i in range(0, len(M_string), block_size)]
    
    # Track the length of the last block
    last_block_length = len(M_string) % block_size
    if last_block_length == 0:
        last_block_length = block_size
    
    print(f"\n🔢 Numeric string: {M_string}")
    print(f"📦 Blocks (size {block_size}): {M_final_blocks}")
    print(f"⚠️  Last block has {last_block_length} digits")
    
    # Encrypt blocks
    print(f"\n🔐 Encrypting blocks:")
    C_blocks = []
    for i, block in enumerate(M_final_blocks, 1):
        C = RSA_encrypt(block)
        C_blocks.append(C)
        print(f"  Block {i}: {block:4d} → {C:4d}")
    
    print(f"\n✅ Ciphertext blocks: {C_blocks}")
    print(f"{'='*60}\n")
    return C_blocks, last_block_length


def RSA_decrypt_plaintext(C_blocks, last_block_length, A, Space):
    """Decrypt ciphertext blocks back to plaintext"""
    print(f"\n{'='*60}")
    print(f"DECRYPTING CIPHERTEXT")
    print(f"{'='*60}")
    print(f"📦 Ciphertext blocks: {C_blocks}")
    print(f"⚠️  Last block should have {last_block_length} digits")
    
    # Decrypt blocks
    print(f"\n🔓 Decrypting blocks:")
    M_blocks = []
    for i, C in enumerate(C_blocks, 1):
        M_block = RSA_decrypt(C)
        # Format last block with correct length
        if i == len(C_blocks):
            M_blocks.append(f"{M_block:0{last_block_length}d}")
        else:
            M_blocks.append(f"{M_block:04d}")
        print(f"  Block {i}: {C:4d} → {M_block:4d}")
    
    # Combine and decode
    M_string = ''.join(M_blocks)
    print(f"\n🔢 Combined numeric string: {M_string}")
    
    print(f"\n📝 Decoding to characters:")
    plaintext = ''
    for i in range(0, len(M_string), 2):
        num = int(M_string[i:i+2])
        if num == Space:
            char = ' '
            print(f"  {num:02d} → ' ' (space)")
        else:
            char = chr(num + A)
            print(f"  {num:02d} → '{char}'")
        plaintext += char
    
    print(f"\n✅ Decrypted plaintext: '{plaintext}'")
    print(f"{'='*60}\n")
    return plaintext



## 4. Testing and Demonstration

In [24]:
print("\n" + "="*60)
print("RSA ENCRYPTION/DECRYPTION DEMONSTRATION")
print("="*60)

# Initialize keys once with larger primes
initialize_keys(p=61, q=53)

# Test 1: Single number encryption
print("\n📌 TEST 1: Single Number Encryption")
print("-" * 60)
message = 88
print(f"Original message: {message}")
ciphertext = RSA_encrypt(message)
print(f"Encrypted: {message} → {ciphertext}")
decrypted = RSA_decrypt(ciphertext)
print(f"Decrypted: {ciphertext} → {decrypted}")
print(f"✓ Success: {message == decrypted}")

# Test 2: Plaintext encryption
print("\n📌 TEST 2: Plaintext Encryption")
print("-" * 60)
plaintext = "MEET ME AFTER"
C, last_block_len = RSA_encrypt_plaintext(plaintext, ord('A'), 26)
decrypted_plaintext = RSA_decrypt_plaintext(C, last_block_len, ord('A'), 26)
print(f"\n🎯 FINAL RESULT:")
print(f"   Original:  '{plaintext}'")
print(f"   Decrypted: '{decrypted_plaintext}'")
print(f"   ✓ Success: {plaintext == decrypted_plaintext}")

# Testing decrypt with a given public key
print("\n📌 TEST 3: Decrypt with Given Public Key")

# Obtain private key from public key with a smaller n for demonstration
private_key = get_private_key(23, 143)
print(f"Derived private key: {private_key}")
public_key = get_public_key(private_key[0], 143)
print(f"Derived public key: {public_key}")
# Set the keys
set_keys(private_key, public_key)
C = 25
decrypted_message = RSA_decrypt(C)
print(f"Decrypted message from C={C} is M={decrypted_message}")

# Verify encryption
C_test = RSA_encrypt(decrypted_message)
print(f"Re-encrypted M={decrypted_message} gives C={C_test}")


RSA ENCRYPTION/DECRYPTION DEMONSTRATION

RSA KEY SETUP
Primes: p = 61, q = 53
Modulus: n = p × q = 3233
Totient: φ(n) = (p-1)(q-1) = 3120

COMPUTING MODULAR INVERSE
Goal: Find x such that (7 × x) ≡ 1 (mod 3120)
      Or equivalently: (7 × x) mod 3120 = 1

Step 1: Use Extended Euclidean Algorithm
        We need 7 × x + 3120 × y = gcd(7, 3120)

🔍 egcd(7, 3120)
  Division: 7 = 3120 × 0 + 7
  Recursing with (3120, 7)...
  🔍 egcd(3120, 7)
    Division: 3120 = 7 × 445 + 5
    Recursing with (7, 5)...
    🔍 egcd(7, 5)
      Division: 7 = 5 × 1 + 2
      Recursing with (5, 2)...
      🔍 egcd(5, 2)
        Division: 5 = 2 × 2 + 1
        Recursing with (2, 1)...
        🔍 egcd(2, 1)
          Division: 2 = 1 × 2 + 0
          Recursing with (1, 0)...
          🔍 egcd(1, 0)
          ✓ Base case: b = 0
            Returning: gcd = 1, x = 1, y = 0
            Verification: 1×1 + 0×0 = 1
        ⬅️  Backtracking from egcd(1, 0)
          Got: g = 1, x1 = 1, y1 = 0
          Computing: x = y1 = 0